In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.fftpack import fft2, ifft2
from scipy.integrate import solve_ivp
from scipy.linalg import solve, lu, solve_triangular
from scipy.sparse.linalg import bicgstab, gmres
from scipy.sparse import csc_matrix
import time
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [2]:
# Define parameters
tspan = np.arange(0, 4.5, 0.5)
nu = 0.001
Lx, Ly = 20, 20
nx, ny = 64, 64
N = nx * ny

In [3]:
# Set initials that we need to build matrix A, B, C
delta =  Lx / nx
e0 = np.zeros((N, 1))  
e1 = np.ones((N, 1))  
e2 = np.copy(e1)   
e4 = np.copy(e0) 

for j in range(1, nx+1):
    e2[nx*j-1] = 0  
    e4[nx*j-1] = 1  

e3 = np.zeros_like(e2)
e3[1:N] = e2[0:N-1]
e3[0] = e2[N-1]

e5 = np.zeros_like(e4)
e5[1:N] = e4[0:N-1]
e5[0] = e4[N-1]

# Build Matrix A
diagonals_A = [e1.flatten(), e1.flatten(), e5.flatten(), 
             e2.flatten(), -4 * e1.flatten(), e3.flatten(), 
             e4.flatten(), e1.flatten(), e1.flatten()]
offsets = [-(N-nx), -nx, -nx+1, -1, 0, 1, nx-1, nx, (N-nx)]

matA = spdiags(diagonals_A, offsets, N, N).toarray()
A = matA / ((delta) ** 2) 

# Build Matrix B
diagonals_B = [e1.flatten(), -1 * e1.flatten(), e1.flatten(), -1 * e1.flatten()]
offsets = [nx, -nx, -N+nx, N-nx]

matB = spdiags(diagonals_B, offsets, N, N).toarray()
B = matB / (2 * delta)

# Build Matrix C
diagonals_C = [e1.flatten(), -e1.flatten()]
offsets_C = [1, -1]

matC = spdiags(diagonals_C, offsets_C, N, N).toarray()

for i in range(nx):
    matC[i * nx, i * nx - 1] = 0
    matC[i * nx - 1, i * nx] = 0
    matC[i * nx, (i + 1) * nx - 1] = -1
    matC[(i + 1) * nx - 1, i * nx] = 1

C = matC / (2 * delta)

# Define spatial domain and initial conditions
x2 = np.linspace(-Lx/2, Lx/2, nx + 1)
x = x2[:nx]
y2 = np.linspace(-Ly/2, Ly/2, ny + 1)
y = y2[:ny]
# initial conditions
X, Y = np.meshgrid(x, y)
w = np.exp(-X**2 - (Y**2 / 20))

# Define spectral k values
kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0, nx/2), np.arange(-nx/2, 0)))
kx[0] = 1e-6
ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0, ny/2), np.arange(-ny/2, 0)))
ky[0] = 1e-6
KX, KY = np.meshgrid(kx, ky)
K = KX**2 + KY**2 

In [4]:
# Define the ODE system
def spc_rhs(t, w0, nx, ny, K, nu):
    w2d = w0.reshape((nx, ny))
    wt = fft2(w2d)
    psit = -wt / K
    psi = np.real(ifft2(psit))
    psi = psi.flatten()
    rhs = (nu * A.dot(w0) + B.dot(w0) * C.dot(psi) - C.dot(w0) * B.dot(psi))
    return rhs

start_time = time.time() # Record the start time
w0 = w.flatten()
wtsol = solve_ivp(spc_rhs, [0, 4], w0, method= "RK45", t_eval=tspan, args=(nx, ny, K, nu))
end_time = time.time() # Record the end time
elapsed_time = end_time - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

A1 = wtsol.y
print(A1)

Elapsed time: 0.66 seconds
[[2.50656748e-46 3.54965186e-45 1.85777180e-44 ... 1.85220714e-42
  4.67310517e-42 1.11018010e-41]
 [1.17762859e-43 6.53203208e-43 2.62143267e-42 ... 1.58860202e-40
  3.56415822e-40 7.46992496e-40]
 [4.55107657e-41 1.92912421e-40 6.43559454e-40 ... 2.49228905e-38
  5.07750991e-38 9.69729364e-38]
 ...
 [1.96785570e-38 1.23335422e-37 5.47293840e-37 ... 4.36630938e-35
  1.02544973e-34 2.23193501e-34]
 [6.19028421e-41 5.33810923e-40 2.93916200e-39 ... 3.69065628e-37
  9.47730687e-37 2.23899842e-36]
 [1.60178709e-43 1.99408907e-42 1.41522826e-41 ... 2.80791338e-39
  7.97536215e-39 2.07833346e-38]]


In [5]:
A1_list = []
for j, t in enumerate(wtsol.t):
    w = A1[:N,j].reshape((nx,ny))
    A1_list.append(w)
fig, ax = plt.subplots()
c = ax.pcolor(X, Y, A1_list[-1], shading='auto', cmap='viridis')
fig.colorbar(c, ax=ax)
time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, color='white', fontsize=12)

# Define the update function for the animation
def update(frame):
    c.set_array(A1_list[frame].flatten())
    time_text.set_text(f'Time: {tspan[frame]:.2f}')
    return c, time_text

# Create the animation
ani = FuncAnimation(fig, update, frames=len(A1_list), interval=100, blit=False)

plt.close(fig)
HTML(ani.to_jshtml())

In [ ]:
# Part b - A\b
def spc_rhs(t, w0, nu):
    psi = solve(A, w0)
    rhs = (nu * A.dot(w0) + B.dot(w0) * C.dot(psi) - C.dot(w0) * B.dot(psi))
    return rhs

A[0, 0] = 2
w = np.exp(-X**2 - Y**2 / 20)
w0 = w.flatten()

start_time = time.time() # Record the start time
wtsol = solve_ivp(spc_rhs, [0, 4], w0, method= "RK45", t_eval=tspan, args=(nu,))
end_time = time.time() # Record the end time
elapsed_time = end_time - start_time
print(f"Elapsed time for A\b: {elapsed_time:.2f} seconds")

A2 = wtsol.y
print(A2)

In [ ]:
A2_list = []
for j, t in enumerate(wtsol.t):
    w = A2[:N,j].reshape((nx,ny))
    A2_list.append(w)
fig, ax = plt.subplots()
c = ax.pcolor(X, Y, A2_list[-1], shading='auto', cmap='viridis')
fig.colorbar(c, ax=ax)
time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, color='white', fontsize=12)

# Define the update function for the animation
def update(frame):
    c.set_array(A2_list[frame].flatten())
    time_text.set_text(f'Time: {tspan[frame]:.2f}')
    return c, time_text

# Create the animation
ani = FuncAnimation(fig, update, frames=len(A2_list), interval=100, blit=False)

plt.close(fig)
HTML(ani.to_jshtml())

In [ ]:
# Part b - Lu
P, L, U = lu(A)
def spc_rhs(t, w0, nu, P, L, U):
    Pb = np.dot(P, w0)
    y = solve_triangular(L, Pb, lower=True)
    psi = solve_triangular(U, y)
    rhs = (nu * A.dot(w0) + B.dot(w0) * C.dot(psi) - C.dot(w0) * B.dot(psi))
    return rhs

w = np.exp(-X**2 - Y**2 / 20)
w0 = w.flatten()

start_time = time.time() # Record the start time
wtsol = solve_ivp(spc_rhs, [0, 4], w0, method= "RK45", t_eval=tspan, args=(nu, P, L, U))
end_time = time.time() # Record the end time
elapsed_time = end_time - start_time
print(f"Elapsed time for LU: {elapsed_time:.2f} seconds")

A3 = wtsol.y
print(A3)

Elapsed time for LU: 1.08 seconds
[[ 2.50656748e-46 -1.78784704e-36  1.02706391e-36 ... -5.76806489e-26
  -6.68287054e-25  3.02050167e-24]
 [ 1.17762859e-43 -2.26093387e-29  1.85614646e-29 ... -1.48306461e-19
  -7.82424220e-19  1.67449651e-18]
 [ 4.55107657e-41 -4.24661881e-29  3.48720993e-29 ... -2.20165937e-20
  -2.91637720e-19  1.23961335e-18]
 ...
 [ 1.96785570e-38  7.20814729e-28 -6.58759786e-28 ...  4.01674620e-20
   1.34371413e-19 -8.94163775e-19]
 [ 6.19028421e-41  1.69117621e-28 -1.49205998e-28 ...  3.00460834e-20
   1.76009486e-19 -9.80736832e-19]
 [ 1.60178709e-43  5.05834665e-29 -4.38046443e-29 ...  2.47633369e-20
   1.15181522e-20  3.12761254e-20]]


In [ ]:
# Part b - BICGSTAB
A_CSC = csc_matrix(A)

def spc_rhs(t, w0, nu):
    psi, exitCode = bicgstab(A_CSC, w0, atol=1e-5)
    rhs = (nu * A.dot(w0) + B.dot(w0) * C.dot(psi) - C.dot(w0) * B.dot(psi))
    return rhs

w = np.exp(-X**2 - Y**2 / 20)
w0 = w.flatten()

start_time = time.time() # Record the start time
wtsol = solve_ivp(spc_rhs, [0, 4], w0, method= "RK45", t_eval=tspan, args=(nu,))
end_time = time.time() # Record the end time
elapsed_time = end_time - start_time
print(f"Elapsed time for BICGSTAB: {elapsed_time:.2f} seconds")

A4 = wtsol.y
print(A4)

Elapsed time for BICGSTAB: 1.35 seconds
[[ 2.50656748e-46 -8.01476113e-37  9.67735928e-37 ... -4.15643332e-25
  -4.35073643e-25  3.18376344e-24]
 [ 1.17762859e-43 -9.44821039e-30  1.36300322e-29 ... -1.14881310e-18
  -4.07435216e-20  1.75690571e-18]
 [ 4.55107657e-41 -1.77339051e-29  2.54578573e-29 ... -1.57810575e-19
  -2.05415919e-19  1.31104955e-18]
 ...
 [ 1.96785570e-38  2.94645665e-28 -4.38533673e-28 ...  3.11741037e-19
  -8.03008630e-20 -9.16755466e-19]
 [ 6.19028421e-41  6.96245697e-29 -1.02084944e-28 ...  2.28758125e-19
   2.52788163e-20 -1.01941479e-18]
 [ 1.60178709e-43  2.09108451e-29 -3.05388361e-29 ...  2.00170057e-19
  -1.32967087e-19  4.57820065e-20]]


In [ ]:
# Part b - GMERS
def spc_rhs(t, w0, nu):
    
    psi, exitCode = gmres(A_CSC, w0, atol=1e-5)
    rhs = (nu * A.dot(w0) + B.dot(w0) * C.dot(psi) - C.dot(w0) * B.dot(psi))
    return rhs

w = np.exp(-X**2 - Y**2 / 20)
w0 = w.flatten()

start_time = time.time() # Record the start time
wtsol = solve_ivp(spc_rhs, [0, 4], w0, method= "RK45", t_eval=tspan, args=(nu,))
end_time = time.time() # Record the end time
elapsed_time = end_time - start_time
print(f"Elapsed time for GMERS: {elapsed_time:.2f} seconds")

A5 = wtsol.y
print(A5)

Elapsed time for GMERS: 15.04 seconds
[[ 2.50656748e-46 -1.78726996e-36  1.02674199e-36 ... -5.76067669e-26
  -6.67857559e-25  3.01864937e-24]
 [ 1.17762859e-43 -2.25993860e-29  1.85529298e-29 ... -1.48130494e-19
  -7.81948820e-19  1.67352252e-18]
 [ 4.55107657e-41 -4.24496803e-29  3.48578542e-29 ... -2.19879556e-20
  -2.91439449e-19  1.23874365e-18]
 ...
 [ 1.96785570e-38  7.20612822e-28 -6.58564566e-28 ...  4.01219584e-20
   1.34316819e-19 -8.93700446e-19]
 [ 6.19028421e-41  1.69064961e-28 -1.49156946e-28 ...  3.00112450e-20
   1.75915606e-19 -9.80151088e-19]
 [ 1.60178709e-43  5.05655804e-29 -4.37883085e-29 ...  2.47412090e-20
   1.15656770e-20  3.11356720e-20]]


In [ ]:
# Part c
def generate_vorticity(scenario):
    if scenario == "opposite":
        w = np.exp(-(X+1)**2 - (Y**2 / 20)) - np.exp(-(X-1)**2 - (Y**2 / 20))
    elif scenario == "same":
        w = np.exp(-(X+1)**2 - (Y**2 / 20)) + np.exp(-(X-1)**2 - (Y**2 / 20))
    elif scenario == "pairs":
        w = np.exp(-(X+1)**2 - ((Y+1)**2 / 20)) - np.exp(-(X-1)**2 - ((Y-1)**2 / 20)) + \
            np.exp(-(X+1)**2 - ((Y-1)**2 / 20)) - np.exp(-(X-1)**2 - ((Y+1)**2 / 20))
    elif scenario == "random":
        num_vortices = 10
        w = np.zeros_like(X) 
        x_positions = np.random.uniform(-Lx/2, Lx/2, num_vortices)
        y_positions = np.random.uniform(-Ly/2, Ly/2, num_vortices)
        amplitudes = np.random.uniform(-1, 1, num_vortices)
        
        for i in range(num_vortices):
            x_i, y_i = x_positions[i], y_positions[i]
            amplitude = amplitudes[i]
            w += amplitude * np.exp(-((X - x_i)**2 + ((Y - y_i)**2) / 20))
    return w


In [ ]:
# Part d
def spc_rhs(t, w0, nx, ny, K, nu):
    w2d = w0.reshape((nx, ny))
    wt = fft2(w2d)
    psit = -wt / K
    psi = np.real(ifft2(psit))
    psi = psi.flatten()
    rhs = (nu * A.dot(w0) + B.dot(w0) * C.dot(psi) - C.dot(w0) * B.dot(psi))
    return rhs


In [ ]:
# Define the scenarios
scenarios = ["opposite", "same", "pairs", "random"]

# Loop through each scenario
for scenario in scenarios:
    print(f"Running scenario: {scenario}")
    
    # Generate initial condition for the current scenario
    w0 = generate_vorticity(scenario).flatten()
    
    # Solve the PDE
    start_time = time.time()
    wtsol = solve_ivp(spc_rhs, [0, 4], w0, method="RK45", t_eval=tspan, args=(nx, ny, K, nu))
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Elapsed time for {scenario}: {elapsed_time:.2f} seconds")
    
    # Plot results for the current scenario
    plt.figure(figsize=(10, 8))
    A6 = []
    for j, t in enumerate(wtsol.t):
        w = wtsol.y[:N,j].reshape((nx,ny))
        A6.append(w)
    fig, ax = plt.subplots()
    c = ax.pcolor(X, Y, A6[-1], shading='auto', cmap='viridis')
    fig.colorbar(c, ax=ax)
    time_text = ax.text(0.05, 0.95, '', transform=ax.transAxes, color='white', fontsize=12)

    # Define the update function for the animation
    def update(frame):
        c.set_array(A6[frame].flatten())
        time_text.set_text(f'Time: {tspan[frame]:.2f}')
        return c, time_text

    # Create the animation
    ani = FuncAnimation(fig, update, frames=len(A6), interval=100, blit=False)

    plt.close(fig)
    HTML(ani.to_jshtml())
    ani.save(f"{scenario}_animation.gif", fps=20, writer="pillow")

Running scenario: opposite
Elapsed time for opposite: 0.52 seconds
Running scenario: same
Elapsed time for same: 0.81 seconds
Running scenario: pairs
Elapsed time for pairs: 0.66 seconds
Running scenario: random
Elapsed time for random: 1.11 seconds


<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>

<Figure size 1000x800 with 0 Axes>